In [1]:
from pft_encoder import read_data, PreFTDataset, Encoder, evaluate_space, pre_fine_tune
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
 

/lv_local/home/omer.y/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = 'Noisy/er_magellan/Structured/Amazon-Google/Amazon-GoogleBert/train.txt'
# path = 'data/er_magellan/Structured/Amazon-Google/train.txt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
left_samples, right_samples, labels = read_data(path)

In [11]:
pft_dataset = PreFTDataset(path, max_len=128)
pft_dataloader = DataLoader(pft_dataset, batch_size=32, shuffle=True)
model = Encoder(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Encoder(
  (transformer): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Lay

In [12]:
pre_fine_tune(model, pft_dataloader, optimizer, criterion, device, epochs=1)


100%|██████████| 215/215 [00:22<00:00,  9.67it/s]

Epoch [1/1], Loss: 0.20280796669336945


In [13]:
positive, negative, pos_centroid, neg_centroid, gap = evaluate_space(model, pft_dataloader, device)

100%|██████████| 215/215 [00:07<00:00, 30.01it/s]


In [14]:
gap

tensor(10.2165, device='cuda:0')